In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# loading data
usc_raw_data = pd.read_csv('/content/drive/MyDrive/Industrial Team Project/USCHAD_rawdata.csv')

In [ ]:
# only features and label.
usc_unprocessed = usc_raw_data[['acc_x','acc_y','acc_z','gyro_x','gyro_y','gyro_z','label']]

In [ ]:
usc_unprocessed.head(5)

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label
0,0.977355,0.125578,-0.072511,-1.934198,4.831090,-3.059582,7
1,0.980789,0.125578,-0.072511,-1.128533,4.806113,-2.639555,7
2,0.980789,0.121959,-0.068851,-1.128533,3.979873,-2.222339,7
3,0.980789,0.121959,-0.072511,-0.725701,2.757195,-3.008654,7
4,0.980789,0.121959,-0.068851,-1.118572,2.743157,-2.989551,7


In [ ]:
# window_size: size of time window
# step: overlapping
# data: dataset
def time_windows(window_size,overlapping,data):
  sigmentation_data_temp = []
  sigmentation_data = []
  sigmentation_label = []
  for i in range(0,len(data),overlapping):
    acc_x = data['acc_x'].values[i:i+window_size]
    acc_y = data['acc_y'].values[i:i+window_size]
    acc_z = data['acc_z'].values[i:i+window_size]
    gyro_x = data['gyro_x'].values[i:i+window_size]
    gyro_y = data['gyro_y'].values[i:i+window_size]
    gyro_z = data['gyro_z'].values[i:i+window_size]
    total_label = data['label'].values[i:i+window_size]
    label = Counter(total_label).most_common()[0][0]
    sigmentation_data_temp.append([acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z])
    sigmentation_arr = np.asarray(sigmentation_data_temp)
    sig_size = sigmentation_arr.shape
    if sig_size[2] == window_size:
      sigmentation_arr.reshape(window_size,6)
      sigmentation_data.append(sigmentation_arr)
      sigmentation_label.append(label)
      sigmentation_data_temp = []
    else:
      sigmentation_data_temp = []

  sigmentation_data_arr = np.asarray(sigmentation_data)
  sigmentation_label_arr = np.asarray(sigmentation_label)

  return sigmentation_data_arr,sigmentation_label_arr


In [ ]:
# here use semi-non-overlapping
# signal segmentation and relevant labels
uci_readings,uci_labels = time_windows(128,64,usc_unprocessed)

In [ ]:
# in the model, requires 0-n label so make labels from 1-12 into 0-11
uci_labels = uci_labels-1

In [ ]:
print('Shape of data:')
print(uci_readings.shape)
print('#######################')
print('Shape of labels: ')
print(uci_labels.shape)

Shape of data:
(43894, 1, 6, 128)
#######################
Shape of labels: 
(43894,)


In [ ]:
# use train_test_split to generate train,val and test data
x_train,x_val_test,y_train,y_val_test = train_test_split(uci_readings,uci_labels,test_size = 0.3)
x_val,x_test,y_val,y_test = train_test_split(x_val_test,y_val_test,test_size = 0.3 )

In [ ]:
# make above variable into tensor
# x_train = torch.Tensor(x_train,dtype = torch.float64)
# y_train = torch.Tensor(y_train,dtype = torch.float64)
# x_val = torch.Tensor(x_val,dtype = torch.float64)
# y_val = torch.Tensor(y_val,dtype = torch.float64)
# x_test = torch.Tensor(x_test,dtype = torch.float64)
# y_test = torch.Tensor(y_test,dtype = torch.float64)

In [ ]:
# use torch.TensorDataset to make dataset
train_dataset = TensorDataset(torch.from_numpy(x_train).to(torch.float32),torch.from_numpy(y_train).to(torch.float32))
val_dataset = TensorDataset(torch.from_numpy(x_val).to(torch.float32),torch.from_numpy(y_val).to(torch.float32))
test_dataset = TensorDataset(torch.from_numpy(x_test).to(torch.float32),torch.from_numpy(y_test).to(torch.float32))

In [ ]:
# DataLoader generation
train_loader = data.DataLoader(dataset = train_dataset,batch_size = 32,shuffle = True)
val_loader = data.DataLoader(dataset = val_dataset,batch_size = 32,shuffle = True)
test_loader = data.DataLoader(dataset = test_dataset,batch_size = 32,shuffle = False)

In [ ]:
# now define a class of CNN model
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU())
    self.drop_out = nn.Dropout()
    self.fc1 = nn.Linear(6144, 120)
    self.fc2 = nn.Linear(120, 12)
  def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
        
model = CNN().to(device)

In [ ]:
# hyperparameters
epoches = 7
lr = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=lr, weight_decay=0.0001)

In [ ]:
# training
loss_list = []
for epoch in range(epoches):
    acc = 0
    for (imgs,labels) in train_loader:
        imgs = imgs.reshape(len(imgs),1,32,24)
        
        imgs = imgs.to(device)
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()
        
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.to(device).long()).sum().item()
        acc = correct / total
    print("Epoch:",epoch+1, "accuracy:", acc)

Epoch: 1 accuracy: 0.8
Epoch: 2 accuracy: 1.0
Epoch: 3 accuracy: 0.8
Epoch: 4 accuracy: 0.6
Epoch: 5 accuracy: 1.0
Epoch: 6 accuracy: 0.8
Epoch: 7 accuracy: 0.2


In [ ]:
# # noise filter.
# from scipy import signal
# # here use bandpass to filter noise from 10-30hz
# current_frequency = 100 # 100hz for the data
# wn1 = 2*10/100
# wn2 = 2*30/100
# b,a = signal.butter(8,[wn1,wn2],'bandpass')
# filtered_data = 

In [ ]:
# gpu   
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")